In [1]:
# Load data (MNIST)

import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

batch_size_train = 100000
batch_size_test = 20000
log_interval = 10


# num_samples = 128 # number of training samples
num_samples = 128
num_samples_test = 100 # number of test samples


old_dim = 28 # MNIST original dimension


random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(root = './', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(root = './', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
    batch_size=batch_size_test, shuffle=True)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

print(example_data.shape)



zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'MNIST.tar.gz'
Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



...

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


torch.Size([10000, 1, 28, 28])


In [2]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

print(example_data.shape)
print(example_targets.shape)

torch.Size([10000, 1, 28, 28])
torch.Size([10000])


In [3]:
# Create training data and test data for binary classification task

from tqdm import tqdm 
training_data = enumerate(train_loader)
test_data = enumerate(test_loader)
(batch_id1, (data_tr_old, target_tr_old)) = next(training_data)
(batch_id2, (data_test_old, target_test_old)) = next(test_data)
print("loaded")


# Classes
class1 = [0]
class2 = [1]

# Pad the image on the right and on the bottom
def filter_digits(data_old, target_old, new_dim, num, threshold):
    data = torch.zeros([num, 1, new_dim, new_dim])
    target = torch.zeros([num])

    num_samples_per_class = int(num / 2)
    sample1 = 0
    sample2 = 0
    attempt = 0
    sample = 0

    while (sample1 < num_samples_per_class) or (sample2 < num_samples_per_class):
        if attempt == threshold:
            print("FAILED: need more samples in batch")
            return (data, target)
        # Balance classes
        is_sample_for_class1 = (target_old[attempt] in class1) and (sample1 < num_samples_per_class)
        is_sample_for_class2 = (target_old[attempt] in class2) and (sample2 < num_samples_per_class)
        if is_sample_for_class1 or is_sample_for_class2: 
            avg = torch.mean(data_old[attempt][0])
            target[sample] = target_old[attempt]
            for i in range(old_dim):
                for j in range(old_dim):
                    data[sample][0][i][j] = data_old[attempt][0][i][j] - avg
            if is_sample_for_class1:
                sample1 += 1
            if is_sample_for_class2:
                sample2 += 1
          # Augment sample counts
            sample += 1
        attempt += 1

    target.apply_(lambda x: 1 if (x in class1) else -1)

    data = data.float()
    target = target.float()
    return (data, target)



loaded


In [ ]:

# Training data and test data for 28 x 28 images 
(data_tr, target_tr) = filter_digits(data_tr_old, target_tr_old, old_dim, num_samples, batch_size_train)
print("Created training data")
(data_test, target_test) = filter_digits(data_test_old, target_test_old, old_dim, num_samples_test, batch_size_test)
print("Created test data")


torch.save(data_tr, "training-test-data/training_data.txt")


torch.save(target_tr, "training-test-data/training_targets.txt")


torch.save(data_test, "training-test-data/test_data.txt")


torch.save(target_test, "training-test-data/test_targets.txt")



In [4]:
# Training data and test data for 112 x 112 padded images 

new_dim = 112 
(data_tr, target_tr) = filter_digits(data_tr_old, target_tr_old, new_dim, num_samples, batch_size_train)
print("Created training data")
(data_test, target_test) = filter_digits(data_test_old, target_test_old, new_dim, num_samples_test, batch_size_test)
print("Created test data")


torch.save(data_tr, "training-test-data/training_data_augmented.txt")


torch.save(target_tr, "training-test-data/training_targets_augmented.txt")


torch.save(data_test, "training-test-data/test_data_augmented.txt")


torch.save(target_test, "training-test-data/test_targets_augmented.txt")


Created training data
Created test data
